# Project Canary: Data Cleaning Pipeline

This notebook cleans and prepares the three data vectors for analysis:
1. **Money Vector**: Lobbying and grant spending data
2. **People Vector**: Energy sector job posting data
3. **Paper Vector**: Regulatory filing activity

## Process Overview
- Load raw CSV data
- Convert dates to datetime format
- Aggregate data by month
- Normalize each metric to 0-1 scale for comparison
- Export cleaned datasets for scoring

In [49]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Money Vector Data

The money vector tracks lobbying and grant spending related to energy policy topics.

In [50]:
# Load money vector data
money_df = pd.read_csv('../data/money_vector.csv')

# Convert date column to datetime
money_df['date'] = pd.to_datetime(money_df['date'])

# Display first few rows
print(f"Money Vector: {len(money_df)} records loaded")
print(f"Date range: {money_df['date'].min()} to {money_df['date'].max()}")
money_df.head()

Money Vector: 220 records loaded
Date range: 2024-11-16 00:00:00 to 2025-10-26 00:00:00


,date,organization,spend_amount,topic
0,2025-10-01,INDIANA HOUSING AND COMMUNITY DEVELOPMENT AUTH...,90475386.0,energy infrastructure
1,2025-09-30,LAKE COUNTY GOVERNMENT CENTER,307890.0,energy infrastructure
2,2025-09-30,STATE OF INDIANA,2448930.0,energy infrastructure
3,2025-09-30,TRUSTEES OF INDIANA UNIVERSITY,4019152.0,energy infrastructure
4,2025-09-30,STATE OF INDIANA,9496380.0,energy infrastructure


## 2. Load People Vector Data

The people vector tracks energy sector job postings as a signal of industry activity.

In [51]:
# Load people vector data
people_df = pd.read_csv('../data/people_vector.csv')

# Convert date column to datetime
people_df['date'] = pd.to_datetime(people_df['date'])

# Display first few rows
print(f"People Vector: {len(people_df)} records loaded")
print(f"Date range: {people_df['date'].min()} to {people_df['date'].max()}")
people_df.head()

People Vector: 329 records loaded
Date range: 2025-07-22 00:00:00 to 2025-10-07 00:00:00


,time,date,job_title,company,location,keywords_detected,source,url
0,16:43.8,2025-10-07,Supervisory Healthcare Engineer (Deputy),Veterans Health Administration,"San Juan, Puerto Rico","doe,energy,epa,utility",usajobs,https://www.usajobs.gov:443/job/847639300
1,41:12.6,2025-10-07,Special Investigative Services Technician (SIS...,"Justice, Bureau of Prisons/Federal Prison System","Seatac, Washington",epa,usajobs,https://www.usajobs.gov:443/job/847631000
2,08:51.3,2025-10-07,Medical Records Technician (Health Information...,Veterans Health Administration,"Bay Pines, Florida","compliance,regulatory",usajobs,https://www.usajobs.gov:443/job/847624800
3,57:23.3,2025-10-07,Supervisory Diagnostic Radiologic Technologist...,Veterans Health Administration,"Cape Coral, Florida",epa,usajobs,https://www.usajobs.gov:443/job/847622800
4,53:39.5,2025-10-07,Supervisory Diagnostic Radiologic Technologist...,Veterans Health Administration,"Cape Coral, Florida",epa,usajobs,https://www.usajobs.gov:443/job/847621900


## 3. Load Paper Vector Data

The paper vector tracks regulatory filings and meeting minutes from energy agencies.

In [52]:
# Load paper vector data
paper_df = pd.read_csv('../data/paper_vector.csv')

# Convert date column to datetime
paper_df['date'] = pd.to_datetime(paper_df['date'])

# Display first few rows
print(f"Paper Vector: {len(paper_df)} records loaded")
print(f"Date range: {paper_df['date'].min()} to {paper_df['date'].max()}")
paper_df.head()

Paper Vector: 442 records loaded
Date range: 2024-10-09 00:00:00 to 2025-10-02 00:00:00


,date,agency,filing_title,keyword_count
0,2024-10-09,Energy Department,Energy Conservation Program for Appliance Stan...,1
1,2024-10-11,Energy Department,Combined Notice of Filings #1,0
2,2024-10-17,Energy Department,Energy Conservation Program: Energy Conservati...,1
3,2024-10-17,Energy Department,Energy Conservation Program for Consumer Produ...,0
4,2024-10-18,Energy Department,Energy Conservation Program: Energy Conservati...,1


## 4. Aggregate Money Vector by Month

Sum total spending per month and normalize to 0-1 scale.

In [53]:
# Extract year-month for grouping
money_df['year_month'] = money_df['date'].dt.to_period('M')

# Aggregate by month: sum of spending
money_monthly = money_df.groupby('year_month').agg({
    'spend_amount': 'sum'
}).reset_index()

# Convert period back to timestamp for compatibility
money_monthly['date'] = money_monthly['year_month'].dt.to_timestamp()

# Normalize to 0-1 scale using min-max normalization
money_min = money_monthly['spend_amount'].min()
money_max = money_monthly['spend_amount'].max()
money_monthly['money_intensity'] = (money_monthly['spend_amount'] - money_min) / (money_max - money_min)

# Keep only necessary columns
money_clean = money_monthly[['date', 'spend_amount', 'money_intensity']]

print(f"Money Vector aggregated to {len(money_clean)} months")
money_clean

Money Vector aggregated to 12 months


,date,spend_amount,money_intensity
0,2024-11-01,3.519660e+04,0.000000
1,2024-12-01,2.906059e+05,0.000401
2,2025-01-01,2.655146e+08,0.417111
3,2025-02-01,3.731839e+05,0.000531
4,2025-03-01,1.603113e+07,0.025132
5,2025-04-01,3.171209e+07,0.049770
6,2025-05-01,1.147764e+08,0.180277
7,2025-06-01,1.049950e+08,0.164909
8,2025-07-01,6.365064e+08,1.000000
9,2025-08-01,3.436096e+07,0.053931


## 5. Aggregate People Vector by Month

Count number of job postings per month and normalize to 0-1 scale.

In [54]:
# Extract year-month for grouping
people_df['year_month'] = people_df['date'].dt.to_period('M')

# Aggregate by month: count of job postings
people_monthly = people_df.groupby('year_month').agg({
    'job_title': 'count'
}).reset_index()

# Rename column for clarity
people_monthly.columns = ['year_month', 'job_count']

# Convert period back to timestamp
people_monthly['date'] = people_monthly['year_month'].dt.to_timestamp()

# Normalize to 0-1 scale
people_min = people_monthly['job_count'].min()
people_max = people_monthly['job_count'].max()
people_monthly['people_intensity'] = (people_monthly['job_count'] - people_min) / (people_max - people_min)

# Keep only necessary columns
people_clean = people_monthly[['date', 'job_count', 'people_intensity']]

print(f"People Vector aggregated to {len(people_clean)} months")
people_clean

People Vector aggregated to 4 months


,date,job_count,people_intensity
0,2025-07-01,2,0.006452
1,2025-08-01,1,0.000000
2,2025-09-01,72,0.458065
3,2025-10-01,156,1.000000


## 6. Aggregate Paper Vector by Month

Sum total keyword mentions per month and normalize to 0-1 scale.

In [55]:
# Extract year-month for grouping
paper_df['year_month'] = paper_df['date'].dt.to_period('M')

# Aggregate by month: sum of keyword counts
paper_monthly = paper_df.groupby('year_month').agg({
    'keyword_count': 'sum'
}).reset_index()

# Convert period back to timestamp
paper_monthly['date'] = paper_monthly['year_month'].dt.to_timestamp()

# Normalize to 0-1 scale
paper_min = paper_monthly['keyword_count'].min()
paper_max = paper_monthly['keyword_count'].max()
paper_monthly['paper_intensity'] = (paper_monthly['keyword_count'] - paper_min) / (paper_max - paper_min)

# Keep only necessary columns
paper_clean = paper_monthly[['date', 'keyword_count', 'paper_intensity']]

print(f"Paper Vector aggregated to {len(paper_clean)} months")
paper_clean

Paper Vector aggregated to 13 months


,date,keyword_count,paper_intensity
0,2024-10-01,6,0.400000
1,2024-11-01,1,0.066667
2,2024-12-01,7,0.466667
3,2025-01-01,7,0.466667
4,2025-02-01,3,0.200000
5,2025-03-01,3,0.200000
6,2025-04-01,7,0.466667
7,2025-05-01,15,1.000000
8,2025-06-01,2,0.133333
9,2025-07-01,1,0.066667


## 7. Summary Statistics

Review the cleaned and normalized data before exporting.

In [56]:
print("=== MONEY VECTOR SUMMARY ===")
print(money_clean.describe())
print("\n=== PEOPLE VECTOR SUMMARY ===")
print(people_clean.describe())
print("\n=== PAPER VECTOR SUMMARY ===")
print(paper_clean.describe())

=== MONEY VECTOR SUMMARY ===
                      date  spend_amount  money_intensity
count                   12  1.200000e+01        12.000000
mean   2025-04-16 16:00:00  1.295356e+08         0.203466
min    2024-11-01 00:00:00  3.519660e+04         0.000000
25%    2025-01-24 06:00:00  1.211664e+07         0.018982
50%    2025-04-16 00:00:00  6.249124e+07         0.098129
75%    2025-07-08 18:00:00  1.508848e+08         0.237009
max    2025-10-01 00:00:00  6.365064e+08         1.000000
std                    NaN  1.849639e+08         0.290608

=== PEOPLE VECTOR SUMMARY ===
                      date   job_count  people_intensity
count                    4    4.000000          4.000000
mean   2025-08-16 06:00:00   57.750000          0.366129
min    2025-07-01 00:00:00    1.000000          0.000000
25%    2025-07-24 06:00:00    1.750000          0.004839
50%    2025-08-16 12:00:00   37.000000          0.232258
75%    2025-09-08 12:00:00   93.000000          0.593548
max    2025-10-01 0

## 8. Export Cleaned Data

Save cleaned and normalized datasets for use in the scoring notebook.

In [57]:
# Note: In a real pipeline, we would export these to CSV files
# For this notebook workflow, we'll pass them directly to the next notebook
# But here's how you would export them:

# money_clean.to_csv('../data/money_clean.csv', index=False)
# people_clean.to_csv('../data/people_clean.csv', index=False)
# paper_clean.to_csv('../data/paper_clean.csv', index=False)

print("✓ Data cleaning complete!")
print(f"✓ Money vector: {len(money_clean)} months with normalized intensity scores")
print(f"✓ People vector: {len(people_clean)} months with normalized intensity scores")
print(f"✓ Paper vector: {len(paper_clean)} months with normalized intensity scores")
print("\nReady for Policy Momentum Score calculation in notebook 2!")

✓ Data cleaning complete!
✓ Money vector: 12 months with normalized intensity scores
✓ People vector: 4 months with normalized intensity scores
✓ Paper vector: 13 months with normalized intensity scores

Ready for Policy Momentum Score calculation in notebook 2!
